In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np

pybamm.set_logging_level('INFO')

In [2]:
parameter_values = pybamm.ParameterValues("Mohtat2020")
parameter_values.update({"SEI kinetic rate constant [m.s-1]": 1e-14})
spm = pybamm.lithium_ion.SPM({"SEI": "ec reaction limited"})

2022-05-01 17:58:24,893 - [INFO] base_battery_model.build_model(834): Start building Single Particle Model
2022-05-01 17:58:24,945 - [INFO] base_battery_model.build_model(854): Finish building Single Particle Model


In [3]:
# Calculate stoichiometries at 100% SOC
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
# esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = spm.param

Vmin = 3.0
Vmax = 4.2
Cn = parameter_values.evaluate(param.C_n_init)
Cp = parameter_values.evaluate(param.C_p_init)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
)
print(f"Initial negative electrode SOC: {esoh_sol['x_100'].data[0]:.3f}")
print(f"Initial positive electrode SOC: {esoh_sol['y_100'].data[0]:.3f}")

# Update parameter values with initial conditions
c_n_max = parameter_values.evaluate(param.c_n_max)
c_p_max = parameter_values.evaluate(param.c_p_max)
parameter_values.update(
    {
        "Initial concentration in negative electrode [mol.m-3]": esoh_sol["x_100"].data[0] * c_n_max,
        "Initial concentration in positive electrode [mol.m-3]": esoh_sol["y_100"].data[0] * c_p_max,
    }
)

NameError: name 'esoh_sim' is not defined

In [ ]:
Cn

In [ ]:
Cp

In [ ]:
sorted(parameter_values.keys())

In [4]:
parameter_values.evaluate(param.C_n)

0.07795937097247523

In [5]:
sorted(spm.variables)

['Ambient temperature',
 'Ambient temperature [K]',
 'Average negative particle concentration',
 'Average negative particle concentration [mol.m-3]',
 'Average positive particle concentration',
 'Average positive particle concentration [mol.m-3]',
 'Battery voltage [V]',
 'C-rate',
 'Cell temperature',
 'Cell temperature [K]',
 'Change in measured open circuit voltage',
 'Change in measured open circuit voltage [V]',
 'Current [A]',
 'Current collector current density',
 'Current collector current density [A.m-2]',
 'Current density variable',
 'Discharge capacity [A.h]',
 'EC surface concentration',
 'EC surface concentration [mol.m-3]',
 'Electrode current density',
 'Electrode tortuosity',
 'Electrolyte concentration',
 'Electrolyte concentration [Molar]',
 'Electrolyte concentration [mol.m-3]',
 'Electrolyte current density',
 'Electrolyte current density [A.m-2]',
 'Electrolyte flux',
 'Electrolyte flux [mol.m-2.s-1]',
 'Electrolyte potential',
 'Electrolyte potential [V]',
 'Elec

In [6]:
parameter_values['Lower voltage cut-off [V]']

2.8

In [10]:
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues('Chen2020')
experiment = pybamm.Experiment([(
    'Discharge at 0.2 C until 2.5 V',
    'Rest for 4 hours',
    'Charge at 0.2 C until 4.2 V',
    'Hold at 4.2 V until C/100',
    'Rest for 2 hours'
)])

simulation = pybamm.Simulation(
    model=model, 
    experiment=experiment, 
    parameter_values=parameter_values, 
    solver=pybamm.CasadiSolver()
)
simulation.solve()

2022-05-01 17:59:25,318 - [INFO] base_battery_model.build_model(834): Start building Doyle-Fuller-Newman model
2022-05-01 17:59:25,378 - [INFO] base_battery_model.build_model(854): Finish building Doyle-Fuller-Newman model
2022-05-01 17:59:25,405 - [INFO] replace_symbols.process_model(47): Start replacing symbols in Doyle-Fuller-Newman model
2022-05-01 17:59:25,434 - [INFO] base_battery_model.build_model(834): Start building Doyle-Fuller-Newman model
2022-05-01 17:59:25,490 - [INFO] base_battery_model.build_model(854): Finish building Doyle-Fuller-Newman model
2022-05-01 17:59:25,532 - [INFO] replace_symbols.process_model(121): Finish replacing symbols in Doyle-Fuller-Newman model
2022-05-01 17:59:25,533 - [INFO] replace_symbols.process_model(47): Start replacing symbols in Doyle-Fuller-Newman model
2022-05-01 17:59:25,561 - [INFO] base_battery_model.build_model(834): Start building Doyle-Fuller-Newman model
2022-05-01 17:59:25,615 - [INFO] base_battery_model.build_model(854): Finish b

In [11]:
simulation.solution['Negative electrode SOC'].entries[-1]

0.9079302819334435

In [12]:
simulation.solution['Discharge capacity [A.h]'].entries[-1]

-0.038071130777077675